# DBLP Publication Visualizer Project

This notebook fetches and visualizes publication data for up to 3 authors using the DBLP API (Method 2). It handles pagination and provides:
- Bar plots: Publications per year
- Line plots: Timeline of publication activity
- Pie charts: Publication type distribution

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# Function to fetch publications for a given author PID (DBLP persistent identifier)
def fetch_publications(pid):
    base_url = f"https://dblp.org/pid/{pid}.json"
    limit = 1000
    offset = 0
    all_publications = []

    while True:
        url = f"{base_url}?offset={offset}&limit={limit}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data for offset {offset}")
            break
        data = response.json()
        hits = data.get("result", {}).get("hits", {}).get("hit", [])
        if not hits:
            break
        for hit in hits:
            info = hit.get("info", {})
            info['type'] = hit.get("type", "unknown")
            all_publications.append(info)
        offset += limit
    return pd.DataFrame(all_publications)

# Function to generate visualizations for an author's publications
def visualize_publications(df, author_name):
    df = df[df['year'].notnull()]
    df['year'] = df['year'].astype(int)

    # Bar Plot: Publications per year
    plt.figure(figsize=(10,5))
    sns.countplot(x="year", data=df, palette="Blues_d")
    plt.title(f"Publications per Year - {author_name}")
    plt.xlabel("Year")
    plt.ylabel("Number of Publications")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Line Plot: Publication activity timeline
    year_counts = df['year'].value_counts().sort_index()
    plt.figure(figsize=(10,5))
    sns.lineplot(x=year_counts.index, y=year_counts.values, marker='o')
    plt.title(f"Publication Activity Timeline - {author_name}")
    plt.xlabel("Year")
    plt.ylabel("Publications")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Pie Chart: Publication type distribution
    plt.figure(figsize=(6,6))
    df['type'].value_counts().plot.pie(autopct='%1.1f%%', startangle=140)
    plt.title(f"Publication Type Distribution - {author_name}")
    plt.ylabel("")
    plt.tight_layout()
    plt.show()

# List of authors with their DBLP PIDs
authors = {
    "Michael Stonebraker": "s/MichaelStonebraker",
    "Geoffrey Hinton": "h/GeoffreyEHinton",
    "Yann LeCun": "l/YannLeCun"
}

# Execute for each author
for name, pid in authors.items():
    print(f"\nFetching publications for {name}...")
    df = fetch_publications(pid)
    print(f"Total publications retrieved: {len(df)}")
    visualize_publications(df, name)
